1) Como vimos em aula, é possível usar Poisson para modelar uma pesca. Suponha que um
pescador pesca por 10 horas com taxa de chegada de peixes λ = 1/h. Agora, suponha também
que, após cada peixe pescado, o pescador interrompe a pesca e gasta meia hora limpando o
peixe antes de retomar a pescaria. a) Plote um exemplo dessa pesca. b) Usando Monte Carlo,
gere um gráfico da probabilidade de cada quantidade de peixes pescados (isto é, um gráfico de
Nº de Peixes X Probabilidade, que mostre qual a probabilidade de a pescaria resultar em 0, 1,
2, 3,... peixes). c) Analisando os resultados anteriores, essa pescaria continua sendo bem
modelada como um processo de Poisson? Justifique.

2) Considere o exercício das 2 máquinas (exercício 4, slides de aula de cadeia de Markov Contínua).
Simule e plote os resultados do tempo que as máquinas passam em cada estado para vários pares
de valores de (λ, μ). Analise e comente os resultados.

3) Considere o exercício do Spa (exercício 5, slides de aula de cadeia de Markov Contínua).

a) Considerando μ1 = μ2 = 1, simule o problema e plote o gráfico do número médio de
pessoas no sistema (L) pela taxa de chegada (λ).

b) Fixando a taxa de chegada em λ = 0.8, simule o problema e plote o gráfico do número
médio de pessoas no sistema (L) e o tempo de espera (W) pelo tamanho permitido da
fila de espera (E). Considere que segundo o enunciado original, a fila de espera do Spa
tem tamanho 0, mas que agora é possível que haja pessoas no Spa aguardando
serviço.

4) Considere uma fila de banco M/M/1 com taxa de chegada λ = 1 e taxa de serviço μ = 1.2.
Considere ainda que cada cliente que chega tem uma probabilidade P de ser um cliente
Preferencial, e que o cliente preferencial passa na frente de todos os clientes regulares.

Sa) Simule e plote o gráfico do tempo médio de espera W por alguns valores de P (inclua
P=0).

b) Simule e plote um boxplot (também chamado de box & whisker plot) do tempo de
espera por alguns valores de P (inclua P=0).

5) Compare o tempo de espera (W) de 2 filas M/M/1 independentes com taxa de chegada λ = 1
cada uma e de 1 fila única M/M/2 com taxa de chegada λ = 2. Calcule o tempo médio que um
atendente fica ocioso em cada caso. Assuma a mesma taxa de serviço para todos os
atendentes.

6) Considere a rede de filas M/M/1 da figura abaixo, representando um trecho de avenidas de
uma grande cidade, com cada fila representando um cruzamento congestionado. Considere os
valores: λ1 = 10, λ2 = 14, μA = 15, μB = 30, μC = 24, μD = 20, μE = 6 (dados em veículos por
minuto). A prefeitura deseja realizar obras para melhorar o fluxo de veículos em um dos 5
cruzamentos (isto é, aumentar a taxa de serviço), visando reduzir o tempo médio de espera
(W) na rede. Mostre por simulação e discuta qual o cruzamento seria o melhor candidato para
a melhoria.